In [1]:
from transformers import Blip2Config, Blip2QFormerConfig, Blip2VisionConfig
from src.models.surroundblip import (
    SurroundBlipForPretraining, SurroundBlipForConditionalGeneration
)
import torch
import torch.nn as nn
from transformers import logging

if __name__ == '__main__':
    import os
    import tempfile
    # 이 스크립트는 개념 증명을 위한 것이며, 실제 데이터로더가 필요합니다.
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # --- 1단계: 표현 학습 ---
    print("="*50)
    print("STAGE 1: Vision-Language Representation Pre-training")
    print("="*50)

    # 1. 모델 및 설정 초기화
    stage1_config = Blip2Config()
    model_stage1 = SurroundBlipForPretraining(stage1_config).to(device)
    optimizer_s1 = torch.optim.AdamW(model_stage1.parameters(), lr=1e-5)

    # 2. 가상 데이터로 1단계 학습 시뮬레이션
    # 실제로는 데이터로더에서 배치 데이터를 가져와야 합니다.
    dummy_pixel_values_s1 = torch.randn(4, 3, 3, 224, 224).to(device) # (B, P, C, H, W)
    dummy_input_ids_s1 = torch.randint(100, 30000, (8, 32)).to(device) # 4개 이미지, 각각 긍정/부정 텍스트 -> 8개
    dummy_attention_mask_s1 = torch.ones_like(dummy_input_ids_s1)
    dummy_labels_itm = torch.tensor([0, 1] * 4).to(device) # [match, no-match] 반복

    model_stage1.train()
    output_s1 = model_stage1(
        pixel_values=dummy_pixel_values_s1,
        input_ids=dummy_input_ids_s1,
        attention_mask=dummy_attention_mask_s1,
        labels_itm=dummy_labels_itm,
        overlap_consistency_weight=0.1
    )
    loss_s1 = output_s1.loss
    # loss_s1.backward() # 실제 학습 시 주석 해제
    # optimizer_s1.step()
    print(f"Stage 1 Loss: {loss_s1.item():.4f}")

    # 3. 학습된 1단계 모델 저장 (임시 디렉터리에)
    with tempfile.TemporaryDirectory() as tmpdir:
        stage1_save_path = os.path.join(tmpdir, "surroundblip_stage1_pretrained")
        model_stage1.save_pretrained(stage1_save_path)
        print(f"Stage 1 model saved to: {stage1_save_path}")

        # --- 2단계: 조건부 생성 학습 ---
        print("\n" + "="*50)
        print("STAGE 2: Conditional Generation Fine-tuning")
        print("="*50)
        
        # 4. 1단계 가중치를 불러와 2단계 모델 초기화
        # strict=False는 1단계 모델에 없는 LLM 가중치는 무시하고 로드하라는 의미
        stage2_config = Blip2Config()
        model_stage2 = SurroundBlipForConditionalGeneration.from_pretrained(
            stage1_save_path, config=stage2_config, strict=False
        ).to(device)
        print("Stage 2 model loaded with pre-trained weights from Stage 1.")

        # 5. 2단계 학습 시뮬레이션
        # 일반적으로 vision_model은 동결하고 qformer와 projection layer만 학습
        for param in model_stage2.vision_model.parameters():
            param.requires_grad = False
        
        optimizer_s2 = torch.optim.AdamW(filter(lambda p: p.requires_grad, model_stage2.parameters()), lr=2e-5)

        dummy_pixel_values_s2 = torch.randn(4, 3, 3, 224, 224).to(device)
        dummy_prompt_ids_s2 = torch.randint(100, 30000, (4, 10)).to(device) # 프롬프트
        dummy_labels_s2 = torch.randint(100, 30000, (4, 50)).to(device)   # 생성할 정답 텍스트

        model_stage2.train()
        output_s2 = model_stage2(
            pixel_values=dummy_pixel_values_s2,
            input_ids=dummy_prompt_ids_s2,
            labels=dummy_labels_s2
        )
        loss_s2 = output_s2.loss
        # loss_s2.backward() # 실제 학습 시 주석 해제
        # optimizer_s2.step()
        print(f"Stage 2 Loss: {loss_s2.item():.4f}")
        
        # 6. 학습된 2단계 모델로 추론(생성) 실행
        model_stage2.eval()
        print("\nRunning generation...")
        generated_ids = model_stage2.generate(
            pixel_values=dummy_pixel_values_s2[:1], # 첫 번째 이미지로 생성
            input_ids=dummy_prompt_ids_s2[:1],      # 첫 번째 프롬프트 사용
            max_length=30
        )
        print(f"Generated token IDs: {generated_ids}")

/Users/seungwoo/anaconda3/envs/sur/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


STAGE 1: Vision-Language Representation Pre-training


VICReg loss calculation failed: shape '[4, 3, 16, 16, 1408]' is invalid for input of size 4342272


TypeError: Blip2QFormerModel.forward() got an unexpected keyword argument 'input_ids'